In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import sk_feature_selection
from sklearn.cluster import KMeans
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
sampled_data = pd.read_csv('../../../data/intermediate/training.csv')

In [4]:
yaml_out_fd = '../Data_prep'

In [5]:
varlist = ['payment_type','employment_status','housing_status','device_os','month']

In [6]:
def cluster_dummy(df, coef_col, varlist, theshold):
    
    newdf = pd.DataFrame(columns = df.columns)

    cluster = {}

    cluster['get_dummies'] = varlist
    
    for var in varlist:
        df_filtered = df[df['feature'].str.contains(var)]
        inertia = []
        models = []
        max_cluster = df_filtered.shape[0]
        
        if max_cluster == 1:
            df_filtered['group'] = [0]
            break
        
        for i in range(2,max_cluster):
            kmeans = KMeans(init="k-means++",n_clusters=i, random_state=0).fit(df_filtered[[coef_col]])
            inertia.append(kmeans.inertia_)
            models.append(kmeans)
            if (kmeans.inertia_ < inertia[-1]) and (inertia[-1] - kmeans.inertia_)/kmeans.inertia_ < threshold: 
                break

        df_filtered['group'] = models[-1].labels_

        cluster[var] = []
        
        for c in range(len(df_filtered['group'].unique())):
        
            cluster[var].append(f'{c},'+ ','.join(df_filtered[df_filtered['group']==c].feature.tolist()))

    return cluster
    

In [7]:
def batch_process_by_segment(seg, varlist):
    segment_data = sampled_data.loc[sampled_data['segment']==seg]
    segment_data_dummy = f_get_dummies(segment_data, varlist=varlist, drop_first=True)
    selector = sk_feature_selection()
    dummy_features = [f for f in segment_data_dummy.columns if (f not in segment_data.columns) and ('nan' not in f)]    
    coef = selector.f_feature_select(segment_data_dummy, dummy_features, 'fraud_bool', mtype='classification', chart='off')
    return segment_data_dummy, coef

In [8]:
segment_data_0, coef0 = batch_process_by_segment(0, varlist)
coef0_clustered = cluster_dummy(coef0, 'linear_SVC', varlist, 0.1)
with open(f'{yaml_out_fd}/dummy_grouping_segment0_draft.yaml', 'w') as outfile:
    yaml.dump(coef0_clustered, outfile,sort_keys=False)

In [9]:
segment_data_1, coef1 = batch_process_by_segment(1, varlist)
coef1_clustered = cluster_dummy(coef1, 'linear_SVC', varlist, 0.1)
with open(f'{yaml_out_fd}/dummy_grouping_segment1_draft.yaml', 'w') as outfile:
    yaml.dump(coef1_clustered, outfile,sort_keys=False)

In [10]:
segment_data_2, coef2 = batch_process_by_segment(2, varlist)
coef2_clustered = cluster_dummy(coef2, 'linear_SVC', varlist, 0.1)
with open(f'{yaml_out_fd}/dummy_grouping_segment2_draft.yaml', 'w') as outfile:
    yaml.dump(coef2_clustered, outfile,sort_keys=False)